In [2]:
import json
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import pandas as pd


In [3]:
# Example file paths (replace with actual paths)
wikidata_entity_label_file = '/home/ashrafs/projects/dataset/FEVER-Processed/WIKIDATA_ENTITY_LABEL_MAPPING_v4.json'
feverous_dev_file = '/home/ashrafs/projects/dragon/data/fever/feverous_dev_labelled_statements.jsonl'
output_enriched_file = '/home/ashrafs/projects/dragon/data/fever/enriched_feverous_dev.jsonl'


In [9]:
import json

def map_entity_labels(entity_label_file, feverous_file, output_file):
    # Step 1: Read the entity-label mapping file
    entity_to_label_map = {}
    with open(entity_label_file, 'r') as file:
        entity_labels = json.load(file)
        for entry in entity_labels:
            if 'label' in entry:
                entity_to_label_map[entry['entity']] = entry['label']

    # Step 2: Read the feverous file and replace entity IDs with labels
    enriched_data = []
    with open(feverous_file, 'r') as file:
        for line in file:
            data = json.loads(line)
            entity_statements = data.get("ENTITY_STATEMENTS", {})

            if entity_statements:  # Check if entity_statements is not None and is a dictionary
                new_entity_statements = {}
                for entity, details in entity_statements.items():
                    entity_label = entity_to_label_map.get(entity, entity)  # Default to entity if no label found
                    new_details = details.copy()
                    
                    # Replace entity IDs in statements with labels
                    new_statements = []
                    for statement in details.get("statements", []):
                        new_statement = [entity_to_label_map.get(e, e) for e in statement]
                        new_statements.append(new_statement)
                    new_details["statements"] = new_statements

                    new_entity_statements[entity_label] = new_details

                data["ENTITY_STATEMENTS"] = new_entity_statements
            enriched_data.append(data)

    # Step 3: Write the enriched data to a new file
    with open(output_file, 'w') as file:
        for entry in enriched_data:
            json.dump(entry, file)
            file.write('\n')

    print("Entity label mapping completed and saved to", output_file)



# This function call is for demonstration. You will need to provide the actual file paths.
map_entity_labels(wikidata_entity_label_file, feverous_dev_file, output_enriched_file)


Entity label mapping completed and saved to /home/ashrafs/projects/dragon/data/fever/enriched_feverous_dev.jsonl
